# Azure AI -agentit Model Context Protocol (MCP) -tuen kanssa - Python

Tämä muistikirja esittelee, kuinka käyttää Azure AI -agentteja Model Context Protocol (MCP) -työkalujen kanssa Pythonissa. Se näyttää, kuinka luoda älykäs agentti, joka voi hyödyntää ulkoisia MCP-palvelimia (kuten Microsoft Learn) parannettujen ominaisuuksien saavuttamiseksi käyttämällä avaimetonta autentikointia.


## Asenna tarvittavat Python-paketit

Ensiksi meidän täytyy asentaa tarvittavat Python-paketit:
- **azure-ai-projects**: Azure AI Projects SDK:n ydin
- **azure-ai-agents**: Azure AI Agents SDK agenttien luomiseen ja hallintaan
- **azure-identity**: Tarjoaa avaimettoman autentikoinnin käyttämällä DefaultAzureCredentialia
- **mcp**: Model Context Protocol -toteutus Pythonille


## Avaimeton todennus - Edut

Tämä muistikirja esittelee **avaimettoman todennuksen**, joka tarjoaa useita etuja:
- ✅ **Ei API-avaimia hallittavana** - Käyttää Azureen perustuvaa identiteettitodennusta
- ✅ **Parannettu turvallisuus** - Ei salaisuuksia tallennettuna koodiin tai konfiguraatiotiedostoihin
- ✅ **Automaattinen tunnistetietojen kierto** - Azure hallitsee tunnistetietojen elinkaaren
- ✅ **Roolipohjainen käyttöoikeuksien hallinta** - Käyttää Azuren RBAC:ia tarkkoihin käyttöoikeuksiin
- ✅ **Moniympäristötuki** - Toimii saumattomasti kehitys- ja tuotantoympäristöissä

`DefaultAzureCredential` valitsee automaattisesti parhaan saatavilla olevan tunnistetietolähteen:
1. **Hallinnoitu identiteetti** (kun suoritetaan Azuren sisällä)
2. **Azure CLI** -tunnistetiedot (paikallisen kehityksen aikana)
3. **Visual Studio** -tunnistetiedot
4. **Ympäristömuuttujat** (jos määritetty)
5. **Interaktiivinen selain** -todennus (varavaihtoehtona)


## Avaimeton todennusasetukset

**Edellytykset avaimettomalle todennukselle:**

### Paikallista kehitystä varten:
```bash
# Install Azure CLI and login
az login
# Verify your identity
az account show
```

### Azure-ympäristöissä:
- Ota käyttöön **järjestelmän määrittämä hallinnoitu identiteetti** Azure-resurssissasi
- Määritä hallinnoidulle identiteetille sopivat **RBAC-roolit**:
  - `Cognitive Services OpenAI User` Azure OpenAI -käyttöä varten
  - `AI Developer` Azure AI -projektien käyttöä varten

### Ympäristömuuttujat (valinnainen):
```python
# These are automatically detected by DefaultAzureCredential
# AZURE_CLIENT_ID=<your-client-id>
# AZURE_CLIENT_SECRET=<your-client-secret>
# AZURE_TENANT_ID=<your-tenant-id>
```

**Ei API-avaimia tai yhteysmerkkijonoja tarvitaan!** 🔐


In [ ]:
! pip install azure-ai-projects -U
! pip install azure-ai-agents==1.1.0b4 -U
! pip install azure-identity -U
! pip install mcp==1.11.0 -U

## Tuo tarvittavat kirjastot

Tuo tarvittavat Python-moduulit:
- **os, time**: Pythonin vakiokirjastot ympäristömuuttujille ja viiveille
- **AIProjectClient**: Pääasiakas Azure AI -projekteille
- **DefaultAzureCredential**: Avaimeton todennus Azure-palveluille
- **MCP-liittyvät luokat**: MCP-työkalujen luomiseen ja hallintaan sekä hyväksyntöjen käsittelyyn


In [ ]:
import os, time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval


## Määritä MCP-palvelimen asetukset

Määritä MCP-palvelimen kokoonpano käyttämällä ympäristömuuttujia, joilla on oletusarvot:
- **MCP_SERVER_URL**: MCP-palvelimen URL-osoite (oletuksena Microsoft Learn API)
- **MCP_SERVER_LABEL**: Tunniste MCP-palvelimen tunnistamiseen (oletuksena "mslearn")

Tämä lähestymistapa mahdollistaa joustavan kokoonpanon eri ympäristöissä.


In [ ]:
mcp_server_url = os.environ.get("MCP_SERVER_URL", "https://learn.microsoft.com/api/mcp")
mcp_server_label = os.environ.get("MCP_SERVER_LABEL", "mslearn")

## Luo Azure AI Project -asiakas (avaimeton todennus)

Alusta Azure AI Project -asiakas käyttäen **avaimetonta todennusta**:
- **endpoint**: Azure AI Foundry -projektin päätepisteen URL-osoite
- **credential**: Käyttää `DefaultAzureCredential()` turvalliseen, avaimettomaan todennukseen
- **Ei API-avaimia vaadita**: Löytää ja käyttää automaattisesti parasta saatavilla olevaa tunnistetta

**Todennusprosessi:**
1. Tarkistaa hallitun identiteetin (Azure-ympäristöissä)
2. Siirtyy Azure CLI -tunnisteisiin (paikallista kehitystä varten)
3. Käyttää muita saatavilla olevia tunnistuslähteitä tarpeen mukaan

Tämä lähestymistapa poistaa tarpeen hallita API-avaimia tai yhteysmerkkijonoja koodissasi.


In [ ]:
project_client = AIProjectClient(
    endpoint="Your Azure AI Foundry Endpoint",
    credential=DefaultAzureCredential(),
)

## Luo MCP-työkalumääritelmä

Luo MCP-työkalu, joka yhdistyy Microsoft Learn MCP-palvelimeen:
- **server_label**: Tunniste MCP-palvelimelle
- **server_url**: MCP-palvelimen URL-päätepiste
- **allowed_tools**: Valinnainen lista, jolla rajoitetaan käytettävissä olevia työkaluja (tyhjä lista sallii kaikki työkalut)

Tämä työkalu mahdollistaa agentin pääsyn Microsoft Learn -dokumentaatioon ja -resursseihin.


In [ ]:
mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
)


## Luo agentti ja suorita keskustelu (avaimeton työnkulku)

Tämä kattava osio esittelee täydellisen **avaimettoman agentin työnkulun**:

1. **Luo AI-agentti**: Määritä agentti GPT-4.1 nano -mallilla ja MCP-työkaluilla
2. **Luo keskusteluketju**: Perusta keskusteluketju viestintää varten
3. **Lähetä viesti**: Kysy agentilta Azure OpenAI:n ja OpenAI:n eroista
4. **Käsittele työkalujen hyväksynnät**: Hyväksy MCP-työkalujen kutsut automaattisesti tarvittaessa
5. **Seuraa suoritusta**: Tarkkaile agentin etenemistä ja hoida tarvittavat toimenpiteet
6. **Näytä tulokset**: Esitä keskustelun ja työkalujen käytön yksityiskohdat

**Avaimettomat ominaisuudet:**
- ✅ **Ei kovakoodattuja salaisuuksia** - Kaikki autentikointi hoidetaan Azure-identiteetillä
- ✅ **Turvallinen oletuksena** - Käyttää roolipohjaista käyttöoikeuksien hallintaa
- ✅ **Yksinkertaistettu käyttöönotto** - Ei vaadi tunnistetietojen hallintaa
- ✅ **Auditointia tukeva** - Kaikki käyttö kirjataan Azure-identiteetin kautta

Agentti käyttää MCP-työkaluja Microsoft Learn -resurssien hyödyntämiseen täysin turvallisesti ilman API-avainten hallintaa.


In [ ]:
with project_client:
    agents_client = project_client.agents

    # Create a new agent with keyless authentication
    # NOTE: To reuse existing agent, fetch it with get_agent(agent_id)
    agent = agents_client.create_agent(
        model="Your Azure OpenAI Model Deployment Name",
        name="my-mcp-agent",
        instructions="You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
        tools=mcp_tool.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
    print(f"MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

    # Create thread for communication
    thread = agents_client.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What's difference between Azure OpenAI and OpenAI?",
    )
    print(f"Created message, ID: {message.id}")

    # KEYLESS APPROACH: Handle tool approvals without hardcoded secrets
    
    # Option 1: Completely keyless (recommended for Azure identity-enabled MCP servers)
    # run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    
    # Option 2: With minimal headers (if MCP server requires specific headers)
    # For demonstration purposes, using a placeholder header
    mcp_tool.update_headers("SuperSecret", "123456")  # Replace with actual auth if needed
    
    # Set approval mode - uncomment next line to disable approval requirement completely
    # mcp_tool.set_approval_mode("never")  # Fully automated, no approval needed
    
    run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        
                        # KEYLESS APPROVAL OPTIONS:
                        
                        # Option 1: No headers (fully keyless)
                        # tool_approvals.append(
                        #     ToolApproval(
                        #         tool_call_id=tool_call.id,
                        #         approve=True,
                        #         headers={}  # No headers needed for keyless
                        #     )
                        # )
                        
                        # Option 2: With headers (if MCP server requires them)
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,  # Uses configured headers if needed
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                agents_client.runs.submit_tool_outputs(
                    thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    print(f"Run completed with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Display run steps and tool calls
    run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)

    # Loop through each step
    for step in run_steps:
        print(f"Step {step['id']} status: {step['status']}")

        # Check if there are tool calls in the step details
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])

        if tool_calls:
            print("  MCP Tool calls:")
            for call in tool_calls:
                print(f"    Tool Call ID: {call.get('id')}")
                print(f"    Type: {call.get('type')}")

        print()  # add an extra newline between steps

    # Fetch and log all messages
    messages = agents_client.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)
    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # Example of dynamic tool management (keyless)
    print(f"\nDemonstrating keyless dynamic tool management:")
    print(f"Current allowed tools: {mcp_tool.allowed_tools}")
    print("✅ All operations completed using keyless authentication!")


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
